## The cost of milk to make cheese
<p>The Coronavirus outbreak has caused major economic disruption. The pandemic's impact on the tourism, travel, and hospitality industries has featured frequently in 2020 news reports, but the economic impact has not been isolated to these industries alone. </p>
<p>The UK dairy industry is one such affected industry and has had to work together to address current market challenges and avoid milk waste. The industry can adapt to the temporary decrease in demand by identifying opportunities for processing the milk into storable products such as butter, cheese, and skimmed milk powder. </p>
<p>The UK Department of Environment, Food, and Rural Affairs has contracted your company to perform a detailed analysis of the UK dairy industry to help them understand the industry's value. </p>
<p>Your team has divided the analysis project into several tasks. One of the tasks is to determine the value of the cheddar cheese industry. The revenue from cheese is much more than, for example, whole milk, but a kilogram of cheese requires almost ten liters of raw milk to make. Your first task is to determine the annual cost of milk needed to produce cheddar cheese over the past five years. </p>
<p>You have been provided with three datasets. <em><strong>Please note that 1000 kg = 1 metric tonne.</strong></em></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/milk_prices.csv - The monthly prices, volume, protein percentage and butterfat percentage of milk </b>
    </div>
Source: <a href="https://environment.data.gov.uk/linked-data/cube/explore?uri=http%3A%2F%2Fenvironment.data.gov.uk%2Flinked-data%2Fcatalog%2Fdatasets%2Fentry%2Fmilk-prices-and-composition-of-milk-annual-statistics&filters-drawer=closed">Defra Data Services Platform</a>
<ul>
    <li><b>Time: </b>The month and year when the value was recorded.</li>
    <li><b>Measure Type: </b>The type of measure recorded.</li>
    <li><b>Price:</b> The price of milk (in Pounds per liter).</li>
    <li><b>Volume:</b> The amount of milk produced (in million liters).</li>
    <li><b>Protein:</b> The protein content of the milk (percentage).</li>
    <li><b>Butterfat:</b> The butterfat content of the milk (percentage).</li>
    <li><b>Unit of Measure:</b> The units specific to the measure type.</li>
</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/Milk_products_production.csv - The monthly production figures of milk, cream and cheddar cheese</b>
    </div>
Source: <a href="https://www.gov.uk/government/statistics/milk-utilisation-by-dairies-in-england-and-wales">GOV.UK</a>
    <ul>
        <li><b>Unnamed: 0: </b>The year and month when the value was recorded.</li>
        <li><b>Liquid Milk Production: </b>The total amount of milk produced (in million liters).</li>
        <li><b>Cream Production: </b>The total amount of cream produced (in million liters).</li>
        <li><b>Cheddar Cheese Production:</b>The total amount of cheddar cheese produced (in thousand tonnes).</li>
    </ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">    
    <div style="font-size:16px"><b>datasets/conversion_factors.xls - Liters of milk used to make one kilogram of product</b>
    </div>
Source: <a href="https://www.gov.uk/government/statistics/milk-utilisation-by-dairies-in-england-and-wales">GOV.UK</a>
    <ul>
        <li><b>Product: </b>The type of product.</li>
        <li><b>Conversion factor (litres/kg):</b> Liters of milk used to make one kilogram of product (in liters/kg).</li>
    </ul>
</div>

Below we are just importing modules. 

In [546]:
import pandas as pd
import numpy as np
import datetime as dt

Below we are reading the files in to pandas dataframes, and getting a little info about them by looking at the 
first few rows. 

In [547]:
milk_prices = pd.read_csv('datasets/milk_prices.csv')
print(milk_prices.head())

milk_products = pd.read_csv('datasets/Milk_products_production.csv')
print(milk_products.head())

conversions = pd.read_excel('datasets/conversion_factors.xls')
print(conversions.head())

      Time Measure type   Price  Volume  Protein  Butterfat   Unit of Measure
0  1986-10        Price  0.1697     NaN      NaN        NaN  Pounds per litre
1  2000-04       Volume     NaN  1177.0      NaN        NaN    Million litres
2  1994-12      Protein     NaN     NaN     3.21        NaN        Percentage
3  1997-10        Price  0.2082     NaN      NaN        NaN  Pounds per litre
4  2002-10      Protein     NaN     NaN     3.41        NaN        Percentage
  Unnamed: 0 Liquid Milk Production Cream Production Cheddar Cheese Production
0        NaN         Million Litres   Million Litres           Thousand tonnes
1     Jan-15                  571,2             25,7                      24,2
2     Feb-15                  524,1             22,5                      21,3
3     Mar-15                  583,0             25,7                      26,2
4     Apr-15                  542,0             26,4                      29,3
                        Product Conversion factor (litres/

Below we are gathering some data about how big our dataframes are and how many NaN values they have. 

In [548]:
df1nan = milk_prices.isna().sum()
print(milk_prices.shape)
print(df1nan)
df2nan = milk_products.isna().sum()
print(milk_products.shape)
print(df2nan)
df3nan = conversions.isna().sum()
print(conversions.shape)
print(df3nan)

(1557, 7)
Time                  0
Measure type          0
Price               946
Volume             1244
Protein            1244
Butterfat          1237
Unit of Measure       0
dtype: int64
(73, 4)
Unnamed: 0                   1
Liquid Milk Production       0
Cream Production             0
Cheddar Cheese Production    0
dtype: int64
(15, 2)
Product                          0
Conversion factor (litres/kg)    0
dtype: int64


Below we turn milk_prices['Time'] and milk_products['Date'] into datetime objects. We also re-label the milk_products columns
to include measurements and then drop the 0-index that originally held them. 

In [549]:
milk_prices['Time'] = pd.to_datetime(milk_prices['Time'], format='%Y-%m')
print(milk_prices.head())
print(milk_prices.dtypes)
print(milk_prices.shape)

milk_products.columns = ['Date', 'Liquid Milk Production (Million Litres)', 'Creme Production (Million Litres)', 
                         'Cheddar Cheese Production (Thousand Tonnes)']
milk_products = milk_products.drop([0])
milk_products['Date'] = pd.to_datetime(milk_products['Date'], format='%b-%y')
print(milk_products.head())
print(milk_products.dtypes)

        Time Measure type   Price  Volume  Protein  Butterfat  \
0 1986-10-01        Price  0.1697     NaN      NaN        NaN   
1 2000-04-01       Volume     NaN  1177.0      NaN        NaN   
2 1994-12-01      Protein     NaN     NaN     3.21        NaN   
3 1997-10-01        Price  0.2082     NaN      NaN        NaN   
4 2002-10-01      Protein     NaN     NaN     3.41        NaN   

    Unit of Measure  
0  Pounds per litre  
1    Million litres  
2        Percentage  
3  Pounds per litre  
4        Percentage  
Time               datetime64[ns]
Measure type               object
Price                     float64
Volume                    float64
Protein                   float64
Butterfat                 float64
Unit of Measure            object
dtype: object
(1557, 7)
        Date Liquid Milk Production (Million Litres)  \
1 2015-01-01                                   571,2   
2 2015-02-01                                   524,1   
3 2015-03-01                                   

Now we make milk_prices only select the columns where Price has a non-null value. 

In [550]:
milk_prices = milk_prices[milk_prices['Price'].notna() == True]
print(milk_prices.head())
print(milk_prices.shape)

         Time Measure type   Price  Volume  Protein  Butterfat  \
0  1986-10-01        Price  0.1697     NaN      NaN        NaN   
3  1997-10-01        Price  0.2082     NaN      NaN        NaN   
16 2009-11-01        Price  0.2487     NaN      NaN        NaN   
24 2008-10-01        Price  0.2736     NaN      NaN        NaN   
26 2019-10-01        Price  0.2940     NaN      NaN        NaN   

     Unit of Measure  
0   Pounds per litre  
3   Pounds per litre  
16  Pounds per litre  
24  Pounds per litre  
26  Pounds per litre  
(611, 7)


Below we are replacing the commas with decimals in milk_products. 

In [551]:
test = isinstance(milk_products['Cheddar Cheese Production (Thousand Tonnes)'], object)
print(test)

True


In [552]:
columns = ['Liquid Milk Production (Million Litres)', 'Creme Production (Million Litres)', 
               'Cheddar Cheese Production (Thousand Tonnes)']


milk_products['Liquid Milk Production (Million Litres)'] = milk_products['Liquid Milk Production (Million Litres)'].str.replace(',', '.', regex=True).astype(float)

milk_products['Creme Production (Million Litres)'] = milk_products['Creme Production (Million Litres)'].str.replace(',', '.', regex=True).astype(float)

milk_products['Cheddar Cheese Production (Thousand Tonnes)'] = milk_products['Cheddar Cheese Production (Thousand Tonnes)'].str.replace(',', '.', regex=True).astype(float)

print(milk_products.head())
print(milk_products.dtypes)

        Date  Liquid Milk Production (Million Litres)  \
1 2015-01-01                                    571.2   
2 2015-02-01                                    524.1   
3 2015-03-01                                    583.0   
4 2015-04-01                                    542.0   
5 2015-05-01                                    571.0   

   Creme Production (Million Litres)  \
1                               25.7   
2                               22.5   
3                               25.7   
4                               26.4   
5                               25.6   

   Cheddar Cheese Production (Thousand Tonnes)  
1                                         24.2  
2                                         21.3  
3                                         26.2  
4                                         29.3  
5                                         31.0  
Date                                           datetime64[ns]
Liquid Milk Production (Million Litres)               float6

Below we are grouping by year and then taking the sum of the cheddar cheese production rows in said year. 

In [553]:
milk_products['Date'] = milk_products['Date'].dt.year
print(milk_products.head())

cheese_sums = []

for year in milk_products['Date']:
    total = milk_products[milk_products['Date'] == year]['Cheddar Cheese Production (Thousand Tonnes)'].sum()
    if total not in cheese_sums:
        cheese_sums.append(total)
    
print(cheese_sums)

   Date  Liquid Milk Production (Million Litres)  \
1  2015                                    571.2   
2  2015                                    524.1   
3  2015                                    583.0   
4  2015                                    542.0   
5  2015                                    571.0   

   Creme Production (Million Litres)  \
1                               25.7   
2                               22.5   
3                               25.7   
4                               26.4   
5                               25.6   

   Cheddar Cheese Production (Thousand Tonnes)  
1                                         24.2  
2                                         21.3  
3                                         26.2  
4                                         29.3  
5                                         31.0  
[302.4, 312.1, 325.4, 332.6, 327.0, 338.20000000000005]


Below we multiply the sums of cheese production per year by 9,500,000. We use 9,500,000 because cheese uses 9.5 litres of milk per kg of cheese made, and the production was measured in thousands of tonnes (1000 * 1000). This lets us know how many litres of milk went into all the cheese for a given year. 

In [554]:
litre_milk_per_year = [i * 9500000 for i in cheese_sums]
print(litre_milk_per_year)

[2872800000.0, 2964950000.0, 3091300000.0, 3159700000.0, 3106500000.0, 3212900000.0000005]


Below we are making it so the values only go out to one decimal point. 

In [555]:
litre_milk_per_year = list(np.around(np.array(litre_milk_per_year), 1))
print(litre_milk_per_year)

[2872800000.0, 2964950000.0, 3091300000.0, 3159700000.0, 3106500000.0, 3212900000.0]


Below we make the Time column a datetime object, subset for only the years that are 2015 or later, and then remove all the Times with NaN. 

In [556]:
milk_prices['Time'] = milk_prices['Time'].dt.year
print(milk_prices.head())

milk_prices['Time'] = milk_prices[milk_prices['Time'] >= 2015]
milk_prices = milk_prices.dropna(subset=['Time'])

print(milk_prices)

    Time Measure type   Price  Volume  Protein  Butterfat   Unit of Measure
0   1986        Price  0.1697     NaN      NaN        NaN  Pounds per litre
3   1997        Price  0.2082     NaN      NaN        NaN  Pounds per litre
16  2009        Price  0.2487     NaN      NaN        NaN  Pounds per litre
24  2008        Price  0.2736     NaN      NaN        NaN  Pounds per litre
26  2019        Price  0.2940     NaN      NaN        NaN  Pounds per litre
      Time Measure type   Price  Volume  Protein  Butterfat   Unit of Measure
26    2019        Price  0.2940     NaN      NaN        NaN  Pounds per litre
451   2020        Price  0.2901     NaN      NaN        NaN  Pounds per litre
481   2020        Price  0.2805     NaN      NaN        NaN  Pounds per litre
506   2020        Price  0.2773     NaN      NaN        NaN  Pounds per litre
515   2020        Price  0.2715     NaN      NaN        NaN  Pounds per litre
542   2020        Price  0.2671     NaN      NaN        NaN  Pounds per litr

Below we first sort the dataframe by year(Time), and then separate milk_prices based on year, stored in the Time column, and then average milk costs for each year. 

In [557]:
milk_prices = milk_prices.sort_values('Time')
print(milk_prices.head())

milk_prices_years = []

for year in milk_prices['Time']:
    total = milk_prices[milk_prices['Time'] == year]['Price'].mean()
    if total not in milk_prices_years: 
        milk_prices_years.append(total)
        

print(milk_prices_years)

      Time Measure type   Price  Volume  Protein  Butterfat   Unit of Measure
1044  2015        Price  0.2646     NaN      NaN        NaN  Pounds per litre
630   2015        Price  0.2363     NaN      NaN        NaN  Pounds per litre
879   2015        Price  0.2505     NaN      NaN        NaN  Pounds per litre
618   2015        Price  0.2343     NaN      NaN        NaN  Pounds per litre
706   2015        Price  0.2383     NaN      NaN        NaN  Pounds per litre
[0.24483333333333335, 0.22630833333333333, 0.2876166666666667, 0.29334166666666667, 0.28900833333333337, 0.2843272727272727]


Below we round the price of milk to the 3rd decimal place. 

In [558]:
milk_prices_years = list(np.around(np.array(milk_prices_years), 3))
print(milk_prices_years)

[0.245, 0.226, 0.288, 0.293, 0.289, 0.284]


Below we put everything into our new dataframe for ease of use. 

In [559]:
annual_cost = {'Date': range(2015, 2021), 'milk_price': milk_prices_years, 'litres': litre_milk_per_year}
annual_cost = pd.DataFrame(annual_cost)
print(annual_cost)

   Date  milk_price        litres
0  2015       0.245  2.872800e+09
1  2016       0.226  2.964950e+09
2  2017       0.288  3.091300e+09
3  2018       0.293  3.159700e+09
4  2019       0.289  3.106500e+09
5  2020       0.284  3.212900e+09


Below we multiply the litres of milk used by the price of milk in each respective year to make a new column, Cost. 

In [560]:
annual_cost['Cost'] = (annual_cost['litres'] * annual_cost['milk_price'])
print(annual_cost)

annual_cost = annual_cost.drop(['milk_price', 'litres'], axis=1)
print(annual_cost)

annual_cost = annual_cost.set_index('Date')

   Date  milk_price        litres         Cost
0  2015       0.245  2.872800e+09  703836000.0
1  2016       0.226  2.964950e+09  670078700.0
2  2017       0.288  3.091300e+09  890294400.0
3  2018       0.293  3.159700e+09  925792100.0
4  2019       0.289  3.106500e+09  897778500.0
5  2020       0.284  3.212900e+09  912463600.0
   Date         Cost
0  2015  703836000.0
1  2016  670078700.0
2  2017  890294400.0
3  2018  925792100.0
4  2019  897778500.0
5  2020  912463600.0


In [561]:
print(annual_cost)

             Cost
Date             
2015  703836000.0
2016  670078700.0
2017  890294400.0
2018  925792100.0
2019  897778500.0
2020  912463600.0
